In [1]:
from hfunc import preprocessing
import tensorflow as tf
import pathlib
import os
import copy
import numpy as np
os.environ['TF_FORCE_GPU_ALLOW_GROWTH'] = 'true'

In [2]:
import tensorflow.keras as keras
from keras.backend import clear_session
import gc

# Reset Keras Session
def reset_keras():
    clear_session()
    print(gc.collect())
    print(gc.collect())

Using TensorFlow backend.


In [3]:
import glob

def remove_logs(log_dir):
    files = glob.glob(log_dir)
    for f in files:
        os.remove(f)

In [4]:
DIR_PATH = pathlib.Path(
    'C:\\Users\\lucas\\Documents\\Masters\\data\\kvasir-dataset-v2'
)

ds = preprocessing.load_dataset_images(DIR_PATH, 224, 224)

for image, label in ds.take(8):
    print("Image shape:", image.numpy().shape)
    print('Label:', label.numpy())

del image, label

train_ds, val_ds, test_ds = preprocessing.train_val_test_split(
    ds,
    0.7,
    0.15
)

cachefile = "C:\\Users\\lucas\\Documents\\Masters\\cache\\kvasir"

train_ds = preprocessing.prepare_for_model_use(
    train_ds,
    cache=cachefile+'_train',
    prefetch=True
)
val_ds = preprocessing.prepare_for_model_use(
    val_ds,
    cache=cachefile+'_val',
    shuffle=False,
    repeat=False
)
test_ds = preprocessing.prepare_for_model_use(
    test_ds,
    cache=False,
    shuffle=False,
    prefetch=False,
    repeat=False
)

Image shape: (224, 224, 3)
Label: [ True False False False False False False False]
Image shape: (224, 224, 3)
Label: [False  True False False False False False False]
Image shape: (224, 224, 3)
Label: [False False  True False False False False False]
Image shape: (224, 224, 3)
Label: [False False False  True False False False False]
Image shape: (224, 224, 3)
Label: [False False False False  True False False False]
Image shape: (224, 224, 3)
Label: [False False False False False  True False False]
Image shape: (224, 224, 3)
Label: [False False False False False False  True False]
Image shape: (224, 224, 3)
Label: [False False False False False False False  True]


In [5]:
ind = 14

In [6]:
ind += 1
np.random.seed(2020)
vgg16 = tf.keras.applications.VGG16(weights=None, classes=8)

In [7]:
vgg16.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 224, 224, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0     

In [8]:
adam = tf.keras.optimizers.Adam(learning_rate=0.01, beta_1=0.8, beta_2=0.9)
sgd = tf.keras.optimizers.SGD(learning_rate=0.001, momentum=0.5, nesterov=True)
vgg16.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
earl_stop = tf.keras.callbacks.EarlyStopping(patience=7, restore_best_weights=True)
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"./logs/model_{ind}")

In [10]:
reset_keras()
if not os.path.isfile('./weights/my_weights.index'):
    hist = vgg16.fit(train_ds, epochs=200, validation_data=val_ds, steps_per_epoch=175, validation_steps=35, callbacks=[tensorboard_callback, earl_stop], use_multiprocessing=True)

6462
0


In [16]:
import copy
def estimate_node_importance(model, tester_model, layer_sizes, tol_low, tol_high, data, folder_path):
    
    l, a = model.evaluate(data, verbose=0, steps=38)
    or_weights = model.get_weights()
    weight_len = len(or_weights) - 3
    amounts = []
    places = []
    
    if folder_path:
        # Retriving Zero Nodes
        try:
            r = open(folder_path+'/zeros.txt', 'r')
            temp_z = [list(map(int,line.split())) for line in r.readlines()]
            r.close()
        except:
            print('No Zero file')
            temp_z = []
        # Retriving Important Nodes
        try:
            r = open(folder_path+'/important.txt', 'r')
            temp_imp = [list(map(int,line.split())) for line in r.readlines()]
        except:
            print("No Important File")
            temp_imp = []
        # Retriving Worse Nodes
        try:
            r = open(folder_path+'/worse.txt', 'r+')
            temp_wr = [list(map(int,line.split())) for line in r.readlines()]
            r.close()
        except:
            print("No Worse File")
            temp_wr = []
    
    for layer, size in enumerate(layer_sizes):
        
        if folder_path and layer < len(temp_z):
            z = temp_z[layer]
            wr = temp_wr[layer]
            imp = temp_imp[layer]
            num_zeros = len(z)
            num_worse = len(wr)
            num_important = len(imp)
            done = (num_zeros + num_worse + num_important)
        else:
            num_zeros, num_worse, num_important = (0, 0, 0)
            z = []
            wr = []
            imp = []
            done = 0
            
        print(f'Layer {len(layer_sizes)-layer}')
        for i in range(size):
            if i == 0:
                if done == size:
                    break
                else:
                    i += done
            w = copy.deepcopy(or_weights)
            w[weight_len - (2*layer+1)][...,i] = 0
            w[weight_len - 2*layer][i] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(data, verbose=0, steps=38)
            change = l - nl
            print(f'Node {i}: {change}')
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
                if folder_path:
                    f = open(folder_path+'/zero.txt', 'a+')
                    f.write(str(i)+" ")
                    f.close()
            elif change > 0:
                num_worse += 1
                wr += [i]
                if folder_path:
                    f = open(folder_path+'/worse.txt', 'a+')
                    f.write(str(i)+" ")
                    f.close()
            else:
                num_important += 1
                imp += [i]
                if folder_path:
                    f = open(folder_path+'/important.txt', 'a+')
                    f.write(str(i)+" ")
                    f.close()
        if folder_path:
            f = open(folder_path+'/zero.txt', 'a+')
            f.write("\n")
            f.close()
            f = open(folder_path+'/worse.txt', 'a+')
            f.write("\n")
            f.close()
            f = open(folder_path+'/important.txt', 'a+')
            f.write("\n")
            f.close()
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))
    
    return amounts, places

In [17]:
if not os.path.isfile('./weights/my_weights.index'):
    vgg16.save_weights('./weights/my_weights')
else:
    vgg16.load_weights('./weights/my_weights')

In [18]:
layer_sizes = [4096, 4096, 512, 512, 512, 512, 512, 512, 256, 256, 256, 128, 128, 64, 64]
vgg16_test = tf.keras.applications.VGG16(weights=None, classes=8)
vgg16_test.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
th = 1e-5
tl = -1e-5

In [19]:
th = 1e-5
tl = -1e-5
reset_keras()
amounts, places = estimate_node_importance(vgg16, vgg16_test, layer_sizes, tl, th, val_ds, 'Estimate')

49
0
No Zero file
No Important File
No Worse File
Layer 15
Node 0: -0.0007110238075256348
Node 1: -4.079937934875488e-05
Node 2: 8.526444435119629e-05
Node 3: 1.996755599975586e-06
Node 4: 0.00017729401588439941
Node 5: 0.00034749507904052734
Node 6: -7.492303848266602e-05
Node 7: 0.001067519187927246
Node 8: 0.000688701868057251
Node 9: 1.2636184692382812e-05
Node 10: -0.00013425946235656738
Node 11: -0.00015115737915039062
Node 12: -0.0007081031799316406
Node 13: -0.000319749116897583
Node 14: 0.0003898441791534424
Node 15: -0.0005415678024291992
Node 16: 4.8220157623291016e-05
Node 17: -0.0009801089763641357
Node 18: 0.00035196542739868164
Node 19: 2.0831823348999023e-05
Node 20: -0.0007262825965881348
Node 21: 0.0006413459777832031
Node 22: -3.1888484954833984e-06
Node 23: 6.175041198730469e-05
Node 24: -8.195638656616211e-06
Node 25: -0.000513613224029541
Node 26: -0.0006526708602905273
Node 27: -0.002091825008392334
Node 28: -0.0011531412601470947
Node 29: 0.00044482946395874023


In [20]:
for i, (nz, nw, ni) in enumerate(reversed(amounts)):
    print(f'######### LAYER {i} #########')
    print("Zero Nodes:", nz)
    print("Worse Nodes:", nw)
    print("Important Nodes:", ni)

######### LAYER 0 #########
Zero Nodes: 14
Worse Nodes: 16
Important Nodes: 34
######### LAYER 1 #########
Zero Nodes: 17
Worse Nodes: 22
Important Nodes: 25
######### LAYER 2 #########
Zero Nodes: 30
Worse Nodes: 42
Important Nodes: 56
######### LAYER 3 #########
Zero Nodes: 34
Worse Nodes: 47
Important Nodes: 47
######### LAYER 4 #########
Zero Nodes: 88
Worse Nodes: 93
Important Nodes: 75
######### LAYER 5 #########
Zero Nodes: 73
Worse Nodes: 98
Important Nodes: 85
######### LAYER 6 #########
Zero Nodes: 79
Worse Nodes: 92
Important Nodes: 85
######### LAYER 7 #########
Zero Nodes: 139
Worse Nodes: 175
Important Nodes: 198
######### LAYER 8 #########
Zero Nodes: 137
Worse Nodes: 193
Important Nodes: 182
######### LAYER 9 #########
Zero Nodes: 84
Worse Nodes: 206
Important Nodes: 222
######### LAYER 10 #########
Zero Nodes: 52
Worse Nodes: 189
Important Nodes: 271
######### LAYER 11 #########
Zero Nodes: 28
Worse Nodes: 202
Important Nodes: 282
######### LAYER 12 #########
Zero Node

In [21]:
def node_pruning(model, tester_model, data, layer_sizes, tol, ignore_cutoff, method='exhaustive', folder_path=None):

    loss, acc = model.evaluate(data, verbose=0, steps=38)
    original = model.get_weights()
    weight_len = len(original) - 3
    bas = [acc]
    bls = [loss]
    best_weights = model.get_weights()
    best_acc = 0
    best_loss = 1e20
    ol = loss
    oa = acc
    amounts = []
    places = []
    
    if folder_path:
        # Retriving Nodes removed
        try:
            r = open(folder_path+'/removed.txt', 'r')
            temp_remove = [list(map(int,line.split())) for line in r.readlines()]
            r.close()
        except:
            print('No node removal file')
            temp_remove = []
        # Retriving loss
        try:
            r = open(folder_path+'/loss.txt', 'r')
            bls = list(map(np.double,r.readline().split()))
            ol = bls[-1]
            r.close()
        except:
            print("No Loss File")
            f_loss = open(folder_path+'/loss.txt', "a+")
            bls = [loss]
            f_loss.write(str(loss)+' ')
            f_loss.close()
        # Retriving accuracy
        try:
            r = open(folder_path+'/accuracy.txt', 'r+')
            bas = r.readline().split()
            bas = list(map(np.double,bas))
            oa = bls[-1]
            r.close()
        except:
            print("No Accuracy File")
            f_acc = open(folder_path+'/accuracy.txt', "a+")
            bas = [acc]
            f_acc.write(str(acc)+' ')
            f_acc.close()
        # Retriving best weights
        try:
            tester_model.load_weights(folder_path+'/weights')
            best_weights = tester_model.get_weights()
        except:
            print('No weights found')
        

    for layer, size in enumerate(layer_sizes):
        if folder_path and layer < len(temp_remove):
            nodes_removed = temp_remove[layer]
            num_removed = len(nodes_removed)
            if layer + 1 < len(temp_remove):
                end_not_reached = False
            else:
                end_not_reached = True
        else:
            num_removed = 0
            nodes_removed = []
            end_not_reached = True
        print(f'Starting removal in Layer {len(layer_sizes) - layer}')
        if method == 'exhaustive':
            current_pos = 0
            best_change = tol
            best_pos = -1
            improved = False
            while end_not_reached or improved:
                if not(end_not_reached):
                    end_not_reached = True
                    improved = False
                    current_pos = 0
                    size -= 1
                    nodes_removed += [best_pos]
                    best_weights[weight_len - (2*layer+1)][...,best_pos] = 0
                    best_weights[weight_len - 2*layer][best_pos] = 0
                    best_pos = -1
                    ol = best_loss
                    oa = best_acc
                    bas += [best_acc]
                    bls += [best_loss]
                    best_change = tol
                    num_removed += 1
                if current_pos in nodes_removed:
                    current_pos += 1
                    if current_pos - num_removed >= size:
                        end_not_reached = False
                    continue
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(data, verbose=0, steps=38)
                if ol - nl >= best_change:
                    best_change = ol - nl
                    best_pos = current_pos
                    improved = True
                    best_acc = na
                    best_loss = nl
                current_pos += 1
                if current_pos - num_removed >= size:
                    end_not_reached = False
        elif method == 'greedy':
            nodes_to_estimate = list(np.arange(size))
            if folder_path and layer < len(temp_remove):
                for i in nodes_removed:
                    nodes_to_estimate.remove(i)
            current_pos = nodes_to_estimate[0]
            idx = 0
            while end_not_reached:
                w = copy.deepcopy(best_weights)
                w[weight_len - (2*layer+1)][...,current_pos] = 0
                w[weight_len - 2*layer][current_pos] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(data, verbose=0, steps=38)
                print(f'Node {current_pos}: {ol - nl}')
                if ol - nl >= tol:
                    oa = na
                    ol = nl
                    size -= 1
                    nodes_removed += [current_pos]
                    nodes_to_estimate.remove(current_pos)
                    best_weights[weight_len - (2*layer+1)][..., current_pos] = 0
                    best_weights[weight_len - 2*layer][current_pos] = 0
                    bas += [oa]
                    bls += [ol]
                    num_removed += 1
                    idx = 0
                    if folder_path:
                        f_rem = open(folder_path+'/removed.txt', "a+")
                        f_loss = open(folder_path+'/loss.txt', "a+")
                        f_acc = open(folder_path+'/accuracy.txt', "a+")
                        
                        f_rem.write(str(current_pos) + ' ')
                        f_loss.write(str(ol) + ' ')
                        f_acc.write(str(oa) + ' ')
                        
                        f_rem.close()
                        f_loss.close()
                        f_acc.close()
                        
                        tester_model.save_weights(folder_path+'/weights')
                elif ol - nl <= ignore_cutoff:
                    size -= 1
                    nodes_to_estimate.remove(current_pos)
                else:
                    idx += 1
                if idx >= size:
                    end_not_reached = False
                    if folder_path:
                        f_rem = open(folder_path+'/removed.txt', "a+")
                        f_rem.write('\n')
                        f_rem.close()
                else:
                    current_pos = nodes_to_estimate[idx]
        print(f'Removed {num_removed} nodes for Layer {len(layer_sizes) - layer}')
        amounts.append(num_removed)
        places.append(nodes_removed)
    
    if folder_path:
        f_rem.close()
        f_loss.close()
        f_acc.close()
    
    return best_weights, bas, bls, amounts, places

In [ ]:
#reset_keras()
best_weights, bas, bls, amounts, places = node_pruning(vgg16, vgg16_test, val_ds, layer_sizes, tl, -3e-4, method='greedy', folder_path='.\Pruning')

Starting removal in Layer 15
Node 0: -0.0002002716064453125
Node 12: -1.3500452041625977e-05
Node 13: -7.888674736022949e-05
Node 15: -0.00035879015922546387
Node 17: -0.00030753016471862793
Node 20: -0.00040093064308166504
Node 25: 4.285573959350586e-05
Node 0: -0.00022727251052856445
Node 12: -3.933906555175781e-05
Node 13: -7.63237476348877e-05
Node 26: -9.033083915710449e-05
Node 27: -0.0009859800338745117
Node 28: -0.0004878044128417969
Node 30: -4.5180320739746094e-05
Node 37: -4.589557647705078e-06
Node 0: -0.00021034479141235352
Node 12: -3.8057565689086914e-05
Node 13: -8.067488670349121e-05
Node 26: -0.0001048743724822998
Node 30: -5.08427619934082e-05
Node 38: 0.0002549588680267334
Node 0: -0.0002008378505706787
Node 12: -4.604458808898926e-05
Node 13: -8.794665336608887e-05
Node 26: -0.00012356042861938477
Node 30: -4.9173831939697266e-05
Node 39: -0.00018206238746643066
Node 43: -0.00033918023109436035
Node 45: -5.117058753967285e-05
Node 50: -0.0003038942813873291
Node 55